# Laboratorio di Big Data - a.a. 2022/2023

Materiale a cura di Roberto Grasso. 

roberto.grasso@phd.unict.it

# Lezione 2
In questa lezione vedremo alcuni tra i più noti algoritmi messi a disposizione dalla libreria __MLlib__. 

__MLlib__ (Machine Learning Library) è una libreria di machine learning per Apache Spark. Essa fornisce una vasta gamma di algoritmi di machine learning scalabili, inclusi modelli di regressione, clustering, classificazione, raccomandazione e altro ancora. Questa libreria supporta anche varie funzionalità per la gestione dei dati, come la trasformazione dei dati in formato vettoriale, la normalizzazione dei dati, la rimozione dei valori mancanti e la selezione delle feature.



Cominciamo importando tutto ciò che ci servirà.

In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, FloatType
from pyspark.ml.feature import StringIndexer, VectorAssembler, PCA, BucketedRandomProjectionLSH
from pyspark.ml.stat import Summarizer, KolmogorovSmirnovTest, Correlation
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import col, collect_list, array_distinct
from pyspark.ml.linalg import Vectors
import pandas as pd

Creiamo una sessione Spark.

In [2]:
# Sessione Spark
spark = SparkSession.builder.appName("LabBigData23").getOrCreate()
sc = spark.sparkContext

/usr/local/lib/python3.9/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/03 16:04:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Eseguiremo buona parte degli algoritmi sul noto dataset __iris__. Esso contiene informazioni su tre varietà di iris (setosa, versicolor e virginica) Ognuna di esse è descritta da quattro attributi: lunghezza del sepalo, larghezza del sepalo, lunghezza del petalo e larghezza del petalo.

Il dataset è composto da 150 osservazioni, dove ogni osservazione rappresenta una singola pianta di iris.

In [3]:
# Definiamo lo schema del dataset
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("sepal_length", FloatType(), False),
    StructField("sepal_width", FloatType(), False),
    StructField("petal_length", FloatType(), True),
    StructField("petal_width", FloatType(), True),
    StructField("species", StringType(), True)
])

# Leggiamo il file csv
iris = spark.read.csv("../data/file5.csv", header=True, schema=schema)

# Visualizziamo qualche informazione
iris.printSchema()
iris.describe(["sepal_length", "sepal_width", "petal_length", "petal_width"]).show()



root
 |-- id: integer (nullable = true)
 |-- sepal_length: float (nullable = true)
 |-- sepal_width: float (nullable = true)
 |-- petal_length: float (nullable = true)
 |-- petal_width: float (nullable = true)
 |-- species: string (nullable = true)

+-------+------------------+-------------------+------------------+------------------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|
+-------+------------------+-------------------+------------------+------------------+
|  count|               150|                150|               150|               150|
|   mean| 5.843333326975505| 3.0540000025431313|3.7586666552225747| 1.198666658103466|
| stddev|0.8280661128539085|0.43359431104332985|1.7644204144315179|0.7631607319020202|
|    min|               4.3|                2.0|               1.0|               0.1|
|    max|               7.9|                4.4|               6.9|               2.5|
+-------+------------------+-------------------+------

Per comodità, facciamo un encoding della colonna `species`.

In [4]:
# Possiamo fare un encoding della colonna "species" usando uno StringIndexer
indexer = StringIndexer(inputCol="species", outputCol="label")

# Aggiungiamo una nuova colonna al DatFrame
iris = indexer.fit(iris).transform(iris)

# Trasformiamo "label" in un intero
iris = iris.withColumn("label", iris["label"].cast(IntegerType()))

# Visualizziamo il DataFrame
iris.show(5)


+---+------------+-----------+------------+-----------+-----------+-----+
| id|sepal_length|sepal_width|petal_length|petal_width|    species|label|
+---+------------+-----------+------------+-----------+-----------+-----+
|  1|         5.1|        3.5|         1.4|        0.2|Iris-setosa|    0|
|  2|         4.9|        3.0|         1.4|        0.2|Iris-setosa|    0|
|  3|         4.7|        3.2|         1.3|        0.2|Iris-setosa|    0|
|  4|         4.6|        3.1|         1.5|        0.2|Iris-setosa|    0|
|  5|         5.0|        3.6|         1.4|        0.2|Iris-setosa|    0|
+---+------------+-----------+------------+-----------+-----------+-----+
only showing top 5 rows



La maggior parte degli algoritmi della libreria __MLlib__ si aspettano gli input in un determinato formato. Le feature sono tipicamente rappresentate da un vettore (`pyspark.ml.linalg.Vectors`) mentre la variabile dipendente (sia essa continua o discreta) è un valore numerico. Abbiamo già fatto l'encoding della specie. Adesso creiamo per ciascuna osservazione un vettore con le feature.

In [5]:
# Poiché partiamo da un DatFrame, possiamo usare un VectorAssembler per creare un vettore di features
vectorAssembler = VectorAssembler(inputCols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol = 'features')

# Abbiamo specificato quali sono le feature che devono essere incluse. Trasformiamo il DataFrame
viris = vectorAssembler.transform(iris)

# Prendiamo solo le colonne che ci interessano (features e label)
viris = viris.select(['features', 'label'])

# Visualizziamo il nuovo DataFrame
viris.show(3, truncate=False)


+----------------------------------------------------------------------------+-----+
|features                                                                    |label|
+----------------------------------------------------------------------------+-----+
|[5.099999904632568,3.5,1.399999976158142,0.20000000298023224]               |0    |
|[4.900000095367432,3.0,1.399999976158142,0.20000000298023224]               |0    |
|[4.699999809265137,3.200000047683716,1.2999999523162842,0.20000000298023224]|0    |
+----------------------------------------------------------------------------+-----+
only showing top 3 rows



## Statistica
La libreria __MLlib__ offre gli strumenti classici per l'analisi statistica. In questa lezione vedremo come calcolare alcune statistiche del primo ordine, come effettuare un test di ipotesi e come calcolare alcuni tra i più noti indici di correlazione.

### Statistiche del primo ordine
Cominciamo con le statistiche del primo ordine. Possiamo velocemente calcolarle utilizzando un oggetto `Summarizer`. Basta specificare la colonna contenente il vettore delle feature e la statisica che si vuole calcolare. Vediamo subito qualche esempio.

In [6]:
# Media
viris.select(Summarizer.mean(viris.features).alias("mean")).show(truncate=False)

+-----------------------------------------------------------------------------+
|mean                                                                         |
+-----------------------------------------------------------------------------+
|[5.8433333269755074,3.0540000025431313,3.7586666552225756,1.1986666581034664]|
+-----------------------------------------------------------------------------+



In [7]:
# Varianza
viris.select(Summarizer.variance(viris.features).alias("variance")).show(truncate=False)

+-----------------------------------------------------------------------------+
|variance                                                                     |
+-----------------------------------------------------------------------------+
|[0.685693487256982,0.18800402656913992,3.1131793988626892,0.5824143027172272]|
+-----------------------------------------------------------------------------+



In [8]:
# Massimo
viris.select(Summarizer.max(viris.features).alias("max")).show(truncate=False)

+-----------------------------------------------------------+
|max                                                        |
+-----------------------------------------------------------+
|[7.900000095367432,4.400000095367432,6.900000095367432,2.5]|
+-----------------------------------------------------------+



In [9]:
# Minimo
viris.select(Summarizer.min(viris.features).alias("min")).show(truncate=False)


+-----------------------------------------------+
|min                                            |
+-----------------------------------------------+
|[4.300000190734863,2.0,1.0,0.10000000149011612]|
+-----------------------------------------------+



È possibile associare un _peso_ a ciasuna osservazione e calcolare, ad esempio, la media pesata.
Vediamo un esempio con un toy dataset.


In [10]:
# Si può scegliere di aggiungere dei pesi alle osservazioni

# Creiamo un nuovo DataFrame giocattolo con sole due colonne: peso e features
example = sc.parallelize([Row(weight=1.0, features=Vectors.dense(1.0, 1.0, 1.0)),
                            Row(weight=0.0, features=Vectors.dense(1.0, 2.0, 3.0))]).toDF()

# Visualizziamo il DataFrame
example.show()

# Calcoliamo la media 
example.select(Summarizer.mean(example.features, example.weight).alias("mean")).show(truncate=False)

# Notiamo che l'osservazione con peso 0 non viene considerata

+------+-------------+
|weight|     features|
+------+-------------+
|   1.0|[1.0,1.0,1.0]|
|   0.0|[1.0,2.0,3.0]|
+------+-------------+

+-------------+
|mean         |
+-------------+
|[1.0,1.0,1.0]|
+-------------+



In [11]:
# Creiamo un altro DataFrame giocattolo con sole due colonne: peso e features
example = sc.parallelize([Row(weight=1.0, features=Vectors.dense(1.0, 1.0, 1.0)),
                            Row(weight=0.5, features=Vectors.dense(1.0, 2.0, 3.0)),
                            Row(weight=0.8, features=Vectors.dense(1.0, 2.0, 3.0))]).toDF()

# Visualizziamo il DataFrame
example.show()

# Calcoliamo la media
example.select(Summarizer.mean(example.features, example.weight).alias("mean")).show(truncate=False)

+------+-------------+
|weight|     features|
+------+-------------+
|   1.0|[1.0,1.0,1.0]|
|   0.5|[1.0,2.0,3.0]|
|   0.8|[1.0,2.0,3.0]|
+------+-------------+

+-----------------------------------------+
|mean                                     |
+-----------------------------------------+
|[1.0,1.565217391304348,2.130434782608696]|
+-----------------------------------------+



### Test di ipotesi
Vediamo come effettuare un test di ipotesi. In particolare, soffermiamoci sul test di __Kolmogorov-Smirnov__ a singolo campione. Questo test ci consente di confrontare una distribuzione empirica con una teorica (che può essere normale, uniforme o di Poisson).

Il test confronta la distribuzione empirica dei dati con quella teorica e calcola la massima differenza assoluta tra le due distribuzioni. La differenza viene espressa come un valore di _statistica test_.

Se il _p-value_ è inferiore a una soglia di significatività prefissata (tipicamente 0.05), allora si può rifiutare l'ipotesi nulla e concludere che il campione di dati non proviene dalla distribuzione teorica specificata. Altrimenti, se il _p-value_ è superiore alla soglia di significatività, si può accettare l'ipotesi nulla e concludere che non ci sono prove sufficienti per rifiutarla.

In sintesi, il test di __Kolmogorov-Smirnov__ è un metodo per verificare se un campione di dati segue una distribuzione specifica ed è comunemente utilizzato per valutare la bontà di adattamento di un modello a una distribuzione dati specifica.

Nel seguente esempio vogliamo confontare il nostro dataset (ogni feature) con la distribuzione normale.

In [12]:
# Definiamo una distribuzione teorica
theoretical_distribution = "norm"

# Definiamo le feature che vogliomo testare
features = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

# Facciamo il test per ciascuna feature
for feature in features: 
    ks_test = KolmogorovSmirnovTest.test(iris, feature, theoretical_distribution)
    
    # Visualizziamo i risultati
    results = ks_test.collect()
    p_value = results[0]["pValue"]
    statistic = results[0]["statistic"]
    print(" ### {} ###\npValue: {}\nstatistic: {}\n".format(feature, p_value, statistic))

 ### sepal_length ###
pValue: 0.0
statistic: 0.999991460101879

 ### sepal_width ###
pValue: 8.58291215877216e-12
statistic: 0.979429887511395

 ### petal_length ###
pValue: 1.43607348235264e-11
statistic: 0.8765328405762314

 ### petal_width ###
pValue: 6.474820679613913e-13
statistic: 0.5398278378685344



### Correlazione
Per concludere la parte sulla statistica vediamo come calcolare la correlazione tra le feature del nostro dataset. La libreria MLlib permette di calcolare due tipi di correlazione: Pearson e Spearman.

Cominciamo calcolando la __correlazione di Pearson__. L'indice di correlazione di Pearson è un numero compreso tra $-1$ e $1$ che indica se due variabili sono correlate, anticorrelate o non sono correlate. Valori prossimi a $-1$ indicano che le variabili sono anticorrelate (quando la prima variabile cresce, la seconda decresce e viceversa). Valori prossimi a $1$ indicano che le variabili sono correlate (quando la prima variabile cresce, anche la seconda cresce e viceversa). Infine, valori prossimi a $0$ indicano che le variabili non sono correlate.


In [13]:
# Calcoliamo la correlazione
pearson_corr = Correlation.corr(viris, "features", method="pearson")

# Visualizziamo i risultati
pearson_corr.show(truncate=False)

23/05/03 16:04:44 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/03 16:04:44 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/05/03 16:04:44 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pearson(features)                                                                                                                                                                                                                                                                    

Cerchiamo di visualizzare meglio i risultati.

In [14]:
# Salviamo i risultati in un DataFrame di pandas
pearson_df = pd.DataFrame(pearson_corr.collect()[0]["pearson(features)"].toArray())

# Arrotondiamo alla seconda cifra decimale
pearson_df = pearson_df.round(2)

# Visualizziamo i risultati
print("### Pearson ###\n", pearson_df)

### Pearson ###
       0     1     2     3
0  1.00 -0.11  0.87  0.82
1 -0.11  1.00 -0.42 -0.36
2  0.87 -0.42  1.00  0.96
3  0.82 -0.36  0.96  1.00


Passiamo adesso alla __correlazione di Spearman__. La correlazione di Pearson funziona quando ci sono pochi outlier, quando le variabili sono distribuite in maniera normale e quando sono legate da una relazione lineare. L'indice di correlazione per ranghi di Spearman risolve questi problemi ordinando tutti i valori e sostituendo ad essi il loro rango (la posizione che essi occupano nella lista ordinata) e calcolando l'indice di Pearson sui ranghi.

In [15]:
# Calcoliamo la correlazione
spearman_corr = Correlation.corr(viris, "features", method="spearman")

# Salviamo i risultati in un DataFrame di pandas
spearmann_df = pd.DataFrame(spearman_corr.collect()[0]["spearman(features)"].toArray())

# Arrotondiamo alla seconda cifra decimale
spearmann_df = spearmann_df.round(2)

# Visualizziamo i risultati
print("### Spearman ###\n", spearmann_df)

### Spearman ###
       0     1     2     3
0  1.00 -0.16  0.88  0.83
1 -0.16  1.00 -0.30 -0.28
2  0.88 -0.30  1.00  0.94
3  0.83 -0.28  0.94  1.00


## Principal Component Analysis
Sfruttiamo gli ultimi esempi per introdurre la __PCA__. La __Principal Component Analysis__ (PCA) è una tecnica di riduzione della dimensionalità utilizzata per identificare i pattern nascosti nei dati, rivelando le relazioni tra le variabili. L'obiettivo della PCA è quello di trasformare un insieme di variabili correlate in un nuovo insieme di variabili non correlate, note come componenti principali. In pratica, la PCA identifica gli assi di maggiore varianza nei dati e proietta i dati su questi assi per ottenere i nuovi set di variabili non correlate.

Nel nostro caso non abbiamo molte variabili (anche se alcune di esse sono correlate). Infatti, il seguente esempio serve solo a mostrarvi come eseguire la PCA con MLlib.

In [16]:
# Definiamo: il numero di componenti, la colonna con le feature e il nome della colonna dove verranno salvate le nuove feature.
pca = PCA(k=2, inputCol="features", outputCol="pca_features")

# Aggiungiamo una colonna al DataFrame. Questa colonna conterrà le nuove feature
model = pca.fit(viris)
transformed = model.transform(viris).select(col("pca_features"))

# Visualizziamo l'output
transformed.show(5, truncate=False)

23/05/03 16:04:56 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/05/03 16:04:56 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
+----------------------------------------+
|pca_features                            |
+----------------------------------------+
|[-2.827135891184163,-5.641330980579563] |
|[-2.7959524725854408,-5.145166936416606]|
|[-2.621523420100777,-5.1773780328276695]|
|[-2.7649058506572253,-5.003599278626428]|
|[-2.7827500765369875,-5.64864822663768] |
+----------------------------------------+
only showing top 5 rows



Ci aspettiamo che le nuove feature non siano correlate. Calcoliamo la correlazione usando entrambi i metodi visti in precedenza.

In [17]:
# Calcoliamo la correlazione
pearson_corr = Correlation.corr(transformed, "pca_features", method="pearson")

# Salviamo i risultati in un DataFrame di pandas
pearson_df = pd.DataFrame(pearson_corr.collect()[0]["pearson(pca_features)"].toArray())

# Arrotondiamo alla seconda cifra decimale
pearson_df = pearson_df.round(2)

# Visualizziamo i risultati
print("### Pearson ###\n", pearson_df)

### Pearson ###
      0    1
0  1.0  0.0
1  0.0  1.0


In [18]:
# Calcoliamo la correlazione
spearman_corr = Correlation.corr(transformed, "pca_features",  method="spearman")

# Salviamo i risultati in un DataFrame di pandas
spearmann_df = pd.DataFrame(spearman_corr.collect()[0]["spearman(pca_features)"].toArray())

# Arrotondiamo alla seconda cifra decimale
spearmann_df = spearmann_df.round(2)

# Visualizziamo i risultati
print("### Spearman ###\n", spearmann_df)

### Spearman ###
       0     1
0  1.00  0.14
1  0.14  1.00


Come previsto, le nuove feature non sono correlate tra loro.

## Locality Sensitive Hashing
Il __Locality Sensitive Hashing__ è una tecnica utilizzata per la ricerca di item simili in dataset di grandi dimensioni. Si effettua un mapping degli item appartenenti ad uno spazio ad alta dimensionalità in un altro spazio a dimensionalità più bassa. L'idea è che, tramite hashing, elementi _simili_ verranno assegnati allo stesso bucket. Questo consente di velocizzare notevolmente il tempo richiesto per la ricerca di item simili, pur mantenendo un tasso di errore basso.

Ci sono diverse varianti dell'LSH. Una di queste è nota come __Bucketed Random Projection (BRP)__. 

Il processo di BRP si svolge in tre fasi:

1. _Generazione delle matrici di proiezione random_. Supponiamo di avere dei dati in uno spazio tridimensionale e di volerli mappare in uno spazio bidimensionale. In questo caso verrà creata una matrice di proizione di dimensione $3 \times 2$. Ciascuna entry della matrice sarà randomica. Ad esempio:
$$
\begin{pmatrix}
10 & 1 & -2
\end{pmatrix}
\cdot
\begin{pmatrix}
1 & -1  \\
1 & 1 \\
-1 & -1
\end{pmatrix}
=
\begin{pmatrix}
13 & -7
\end{pmatrix}
$$

2. _Proiezione dei dati_. I vettori originali vengono moltiplicati per la matrice di proiezione. In questo modo vengono mappati in uno spazio a dimensionalità più bassa.

3. _Bucketing_. I vettori proiettati vengono raggruppati in bucket.




Vediamo un esempio.

In [19]:
# Facciamo un Bucketed Random Projection LSH con 3 tabelle hash
brp_lsh = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", numHashTables=3, bucketLength=2.0)
brp_lsh.setSeed(10)

# Applichiamo il modello al nostro dataset
model = brp_lsh.fit(viris)
transformed = model.transform(viris)

# Prendiamo un item a caso 
random_row = viris.sample(False, 0.5, seed=100).take(1)[0]

# Visualizziamolo
print(random_row)

# Cerchiamo i suoi 5 vicini più prossimi
print(model.approxNearestNeighbors(transformed, random_row.features, 5).collect())


Row(features=DenseVector([4.9, 3.0, 1.4, 0.2]), label=0)
[Row(features=DenseVector([4.9, 3.0, 1.4, 0.2]), label=0, hashes=[DenseVector([0.0]), DenseVector([2.0]), DenseVector([-1.0])], distCol=0.0), Row(features=DenseVector([4.8, 3.0, 1.4, 0.1]), label=0, hashes=[DenseVector([0.0]), DenseVector([2.0]), DenseVector([-1.0])], distCol=0.1414212898560397), Row(features=DenseVector([4.8, 3.0, 1.4, 0.3]), label=0, hashes=[DenseVector([0.0]), DenseVector([2.0]), DenseVector([-1.0])], distCol=0.1414212951243984), Row(features=DenseVector([4.9, 3.1, 1.5, 0.1]), label=0, hashes=[DenseVector([0.0]), DenseVector([2.0]), DenseVector([-1.0])], distCol=0.1732050403219206), Row(features=DenseVector([4.9, 3.1, 1.5, 0.1]), label=0, hashes=[DenseVector([0.0]), DenseVector([2.0]), DenseVector([-1.0])], distCol=0.1732050403219206)]


## Clustering
Il __clustering__ è una tecnica di analisi dei dati utilizzata per identificare gruppi omogenei all'interno di un dataset. L'obiettivo del clustering è quello di organizzare gli elementi di un dataset in un certo numero di gruppi, detti cluster, tali che gli oggetti all'interno di ciascun cluster sono più simili tra loro rispetto agli oggetti in altri cluster. 

### K-Means

Vediamo un esempio con uno degli algortimi di clustering più famosi: il __K-Means__. Il K-Means è un algoritmo di clustering semi-supervisionato che cerca di dividere un insieme di dati in K cluster (K è un  parametro dell'algoritmo). L'obiettivo è di minimizzare la somma delle distanze quadrate tra ogni punto e il suo centroide di appartenenza.

L'algoritmo inizia con la scelta di K centroidi casuali all'interno del dataset. In seguito, per ogni punto viene calcolata la distanza euclidea tra esso e ogni centroide. Il punto viene quindi assegnato al cluster il cui centroide è più vicino. Una volta che tutti i punti sono stati assegnati ai cluster, i centroidi vengono aggiornati (media di tutti i punti appartenenti al cluster).

Il processo di assegnazione dei punti ai cluster e di aggiornamento dei centroidi viene ripetuto fino a quando non si raggiunge un criterio di convergenza. Tipicamente, questo criterio è un limite massimo sul numero di iterazioni o un valore minimo sulla variazione della somma delle distanze quadrate tra i centroidi e i punti del cluster.

In [20]:
# Eseguiamo l'algoritmo K-means con k = 3
kmeans = KMeans(k=3, seed=1)
model = kmeans.fit(viris.select("features"))
predictions = model.transform(viris.select("features", "label"))

# Vediamo i  risultati
predictions.show(3, truncate=False)

# Vediamo il numero di elementi di ciascuna classe all'interno dei 3 cluster
class_counts = predictions.groupBy("prediction", "label").count().orderBy("prediction", "label")
class_counts.show()

+----------------------------------------------------------------------------+-----+----------+
|features                                                                    |label|prediction|
+----------------------------------------------------------------------------+-----+----------+
|[5.099999904632568,3.5,1.399999976158142,0.20000000298023224]               |0    |1         |
|[4.900000095367432,3.0,1.399999976158142,0.20000000298023224]               |0    |1         |
|[4.699999809265137,3.200000047683716,1.2999999523162842,0.20000000298023224]|0    |1         |
+----------------------------------------------------------------------------+-----+----------+
only showing top 3 rows

+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|         0|    1|   48|
|         0|    2|   14|
|         1|    0|   50|
|         2|    1|    2|
|         2|    2|   36|
+----------+-----+-----+



## Classificazione


La __classificazione__ è una tecnica di analisi dei dati supervisionata che consiste nell'assegnare degli oggetti a una o più categorie (classi) in base alle loro feature. Il processo di classificazione richiede un dataset, noto come training set, che verrà utilizzato per addestrare un modello e un altro dataset, noto come test set (diverso dal training set), che verrà utilizzato per testarlo.

Dividiamo il nostro dataset in due parti: una per il training e una per il test.

In [21]:
(training_data, test_data) = viris.randomSplit([0.8, 0.2], seed=100)

### Decision Tree
Il primo metodo per la classificazione che vedremo è quello dei __Decision Tree__. Un albero decisionale è un modello di apprendimento supervisionato che può essere utilizzato sia per la classificazione che per la regressione. In pratica, un albero decisionale costruisce una serie di regole che vengono applicate in maniera gerarchica al fine di prevedere una classe (nel caso della classificazione) o un valore continuo (nel caso della regressione).

Il decision tree è un modello di facile interpretazione e può essere visualizzato graficamente, rendendolo adatto a situazioni in cui è necessario spiegare le decisioni adottate dal modello a persone non esperte. Inoltre, può gestire sia variabili di input continue che discrete.

In [23]:
# Addestriamo un Decision Tree
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
model = dt.fit(training_data)

# Facciamo delle previsioni sul test set
predictions = model.transform(test_data)

# Visualizziamo qualche previsione
predictions.select("label", "prediction").show(3, truncate=False)

# Calcoliamo e visualizziamo l'accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print('Accuracy = {:.2f}'.format(accuracy))


+-----+----------+
|label|prediction|
+-----+----------+
|0    |0.0       |
|0    |0.0       |
|0    |0.0       |
+-----+----------+
only showing top 3 rows

Accuracy = 0.90


### Random Forest
Continuiamo con le __Random Forest__. Le Random Forest non fanno altro che allenare molti alberi decisionali (con una profondità limitata) in cui le soglie per lo splitting sono scelte a caso. Una volta che tutti gli alberi sono stati costruiti, la Random Forest combina le loro predizioni per determinare l'output finale. In caso di problemi di classificazione, la foresta restituisce la classe più frequente tra tutte le predizioni degli alberi. In caso di problemi di regressione, la foresta restituisce la media delle predizioni degli alberi.

In [22]:
# Addestriamo una Random Forest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)
model = rf.fit(training_data)

# Facciamo delle previsioni sul test set
predictions = model.transform(test_data)

# Visualizziamo qualche previsione
predictions.select("label", "prediction").show(3, truncate=False)

# Calcoliamo e visualizziamo l'accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print('Accuracy = {:.2f}'.format(accuracy))

+-----+----------+
|label|prediction|
+-----+----------+
|0    |0.0       |
|0    |0.0       |
|0    |0.0       |
+-----+----------+
only showing top 3 rows

Accuracy = 0.97


### Logistic Regression
Concludiamo con la __Logistic Regression__. La regressione logistica è un algoritmo di apprendimento supervisionato utilizzato per la classificazione. L'algoritmo di regressione logistica utilizza la funzione logistica (o sigmoide) per stimare una  probabilità, che può assumere un valore compreso tra 0 e 1. Questa probabilità viene utilizzata per classificare i dati, utilizzando una soglia di decisione (ad esempio, se la probabilità è maggiore di 0,5, l'item verrà assegnato alla classe 1, altrimenti alla classe 0).

In [23]:
# Carichiamo il dataset
breast_cancer_data = spark.read.csv("../data/file8.csv", header=True, inferSchema=True)

# Trasformiamo la colonna con la diagnosi in modo che contenga valori numerici (0 o 1)
indexer = StringIndexer(inputCol="diagnosis", outputCol="label")
breast_cancer_data = indexer.fit(breast_cancer_data).transform(breast_cancer_data)

# Per ciascun elemento, raggruppiamo le feature in un vettore
assembler = VectorAssembler(inputCols=breast_cancer_data.columns[2:], outputCol="features")
vbreast_cancer_data = assembler.transform(breast_cancer_data).select(col("label"), col("features"))

# Visualizziamo qualche elemento
vbreast_cancer_data.show(5, truncate=False)

23/05/03 16:05:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                                                |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1.0  |[17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.0

In [24]:
# Anche in questo caso, suddividiamo il dataset in training e test
(training_data, test_data) = vbreast_cancer_data.randomSplit([0.8, 0.2], seed=100)

In [25]:
# Creiamo il modello per la regressione logistica
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Addestriamo il modello sul training set
model = lr.fit(training_data)

# Facciamo delle previsioni sul test set
predictions = model.transform(test_data)

# Visualizziamo qualche previsione
predictions.select("label", "prediction", "probability").show(3, truncate=False)

+-----+----------+-------------------------------------------+
|label|prediction|probability                                |
+-----+----------+-------------------------------------------+
|0.0  |0.0       |[0.9999999997663465,2.336535409597218E-10] |
|0.0  |0.0       |[0.9999999999880376,1.1962431045731137E-11]|
|0.0  |0.0       |[0.9999999999993061,6.938893903907228E-13] |
+-----+----------+-------------------------------------------+
only showing top 3 rows



In [26]:
# Calcoliamo e visualizziamo l'AUC (Area Under the Curve)
evaluator = BinaryClassificationEvaluator()
auc = evaluator.evaluate(predictions)
print('AUC = {:.2f}'.format(auc))

AUC = 1.00


In [27]:
# Calcoliamo e visualizziamo l'accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print('Accuracy = {:.2f}'.format(accuracy))

Accuracy = 1.00


In [28]:
# Calcoliamo e visualizziamo la precision
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator.evaluate(predictions)
print('Precision = {:.2f}'.format(precision))

Precision = 1.00


In [29]:
# Calcoliamo e visualizziamo la recall
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator.evaluate(predictions)
print('Recall = {:.2f}'.format(recall))

Recall = 1.00


In [30]:
# Calcoliamo e visualizziamo l'F1 score
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)
print('F1 = {:.2f}'.format(f1))

F1 = 1.00


## Regressione
La __regressione__ è una tecnica che viene utilizzata per esplorare la relazione tra una o più variabili indipendenti e una variabile dipendente. Lo scopo della regressione è quello di trovare una funzione matematica che possa descrivere la relazione tra queste variabili.

Carichiamo un dataset più adatto ad un algoritmo di regressione.

In [31]:
# Carichiamo il dataset
wine_data = spark.read.csv("../data/file9.csv", header=True, inferSchema=True)

# Visualizziamo qualche elemento
wine_data.show(5, truncate=False)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|pH  |sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|7.4          |0.7             |0.0        |1.9           |0.076    |11.0               |34.0                |0.9978 |3.51|0.56     |9.4    |5      |
|7.8          |0.88            |0.0        |2.6           |0.098    |25.0               |67.0                |0.9968 |3.2 |0.68     |9.8    |5      |
|7.8          |0.76            |0.04       |2.3           |0.092    |15.0               |54.0                |0.997  |3.26|0.65     |9.8    |5      |
|11.2         |0.28            |0.56       |1.9           |0.075    |17.0               |60.0       

Vogliamo provare a predire la qualità del vino a partire dalle feature che abbiamo a disposizione. Come visto in precedenza, dobbiamo raggruppare le feature di ciascun elemento in un vettore.

In [32]:
# Per ciascun elemento, raggruppiamo le feature in un vettore
assembler = VectorAssembler(inputCols=wine_data.columns[:-1], outputCol="features")
vwine_data = assembler.transform(wine_data).select(col("quality").alias("target"), col("features"))

# Visualizziamo qualche elemento
vwine_data.show(5, truncate=False)


+------+--------------------------------------------------------+
|target|features                                                |
+------+--------------------------------------------------------+
|5     |[7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4]  |
|5     |[7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8]  |
|5     |[7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8] |
|6     |[11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8]|
|5     |[7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4]  |
+------+--------------------------------------------------------+
only showing top 5 rows



Dividiamo il dataset in training e test set.

In [33]:
# Anche in questo caso, suddividiamo il dataset in training e test
(training_data, test_data) = vwine_data.randomSplit([0.8, 0.2])

### Regressione Lineare
Esistono vari tipi di regressione. Uno tra questi è la __regressione lineare__. Questo tipo di regressione prevede l'andamento di una variabile dipendente a partire da uno o più variabili dipendenti assumendo che tale relazione sia lineare.

In [34]:
# Creiamo il modello per la regressione lineare
lr = LinearRegression(featuresCol="features", labelCol="target", regParam=0.1, elasticNetParam=0.5)

# Addestriamo il modello sul training set
model = lr.fit(training_data)

# Facciamo delle previsioni sul test set
predictions = model.transform(test_data)

# Calcoliamo e visualizziamo il Root Mean Squared Error
evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print('RMSE = {:.2f}'.format(rmse))


RMSE = 0.69


## Collaborative filtering

Il __Collaborative Filtering__ è una tecnica utilizzata nei sistemi di raccomandazione per trovare le relazioni tra gli utenti e gli item (come ad esempio i film in una piattaforma di streaming) sulla base delle loro attività passate (come ad esempio i rating dati dagli utenti ai film).

Esistono due tipi principali di Collaborative Filtering: item-based e user-based. Nel Collaborative Filtering user-based, le preferenze degli utenti sono utilizzate per trovare utenti simili, e queste similarità sono utilizzate per fare raccomandazioni sugli item. Nel Collaborative Filtering item-based, invece, le proprietà degli item sono utilizzate per trovare item simili, e queste similarità sono utilizzate per fare raccomandazioni agli utenti.

Il Collaborative Filtering utilizza una matrice di associazione user-item, che contiene, ad esempio, i rating dati dagli utenti agli item. Tuttavia, questa matrice spesso contiene molte voci mancanti (poiché gli utenti non hanno valutato tutti gli item). Il Collaborative Filtering utilizza il machine learning per trovare le relazioni tra gli utenti e gli item. In MLlib, l'algoritmo proposto per il Collaborative Filtering è l'__ALS__ (Alternating Least Square). 

Cominciamo caricando un dataset adatto a questo genere di task.

In [35]:
# Carichiamo il dataset
recc_data = spark.read.csv("../data/file6.csv", inferSchema=True, header=True)

# Stampiamo qualche elemento
recc_data.show(5, truncate=False)

# Vediamo qualche informazione sui rating
recc_data.select("rating").describe().show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|2      |3.0   |0     |
|3      |1.0   |0     |
|5      |2.0   |0     |
|9      |4.0   |0     |
|11     |1.0   |0     |
+-------+------+------+
only showing top 5 rows

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|              1501|
|   mean|1.7741505662891406|
| stddev| 1.187276166124803|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



Dividiamo il dataset in training e test set.

In [36]:
# Anche in questo caso, suddividiamo il dataset in training e test
(training_data, test_data) = recc_data.randomSplit([0.8, 0.2], seed=100)

In [38]:
# Creiamo e addestriamo il modello per il collaborative filtering
als = ALS(maxIter=5, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training_data)

# Vediamo le migliori 5 valutazioni date da alcuni utenti
userRecs = model.recommendForAllUsers(5)
userRecs.show(3, truncate=False)

# Veediamo quali sono le 5 migliori valutazioni date ad alcuni film
movieRecs = model.recommendForAllItems(5)
movieRecs.show(3, truncate=False)

23/05/03 16:06:38 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+------+-----------------------------------------------------------------------------------+
|userId|recommendations                                                                    |
+------+-----------------------------------------------------------------------------------+
|20    |[{22, 3.8885803}, {94, 3.202127}, {77, 3.123834}, {88, 3.0215375}, {51, 2.954056}] |
|10    |[{92, 3.484316}, {2, 3.3816395}, {40, 2.8593414}, {25, 2.7985897}, {49, 2.7787867}]|
|0     |[{92, 3.2650762}, {2, 2.8636913}, {25, 2.670202}, {12, 2.4387572}, {49, 2.2559018}]|
+------+-----------------------------------------------------------------------------------+
only showing top 3 rows

+-------+-----------------------------------------------------------------------------------+
|movieId|recommendations                                                                    |
+-------+-----------------------------------------------------------------------------------+
|20     |[{23, 2.299555}, {17, 2.2393234},

In [39]:
# Raccomandiamo 10 film ad alcuni utenti del test set
users = test_data.select(als.getUserCol()).distinct().limit(3)
users_rec = model.recommendForUserSubset(users, 10)

# Visualizziamo le previsioni
users_rec.show(truncate=False)

# Vediamo quali sono gli utenti del test set a cui potrebbero piaceere alcuni film
movies = training_data.select(als.getItemCol()).distinct().limit(3)
movies_rec = model.recommendForItemSubset(movies, 10)

movies_rec.show(truncate=False)

+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                        |
+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|26    |[{22, 4.589559}, {7, 4.423425}, {88, 4.397339}, {23, 4.304868}, {24, 4.2404084}, {68, 3.6774354}, {36, 3.482273}, {30, 3.4684234}, {4, 3.3449988}, {51, 3.3309832}]    |
|27    |[{30, 3.4716766}, {18, 3.412661}, {23, 2.9196398}, {32, 2.8294592}, {69, 2.7674785}, {7, 2.7615361}, {80, 2.761163}, {66, 2.6972814}, {83, 2.5878484}, {27, 2.5506296}]|
|28    |[{92, 4.400953}, {81, 3.9778917}, {2, 3.8073535}, {12, 3.722697}, {49, 3.4508216}, {4, 3.1810398}, {82, 2.9

# Frequent Itemsets Mining & Association Rules
Il __Frequent Itemset Mining__ è una tecnica di data mining utilizzata per trovare insiemi di item (articoli, prodotti, ecc.) molto frequenti all'interno di un dataset. Ad esempio, in un negozio, ci possono essere prodotti che vengono spesso acquistati insieme e l'identificazione di tali associazioni può essere utile per la strategia di marketing e per le raccomandazioni ai clienti.

La tecnica si basa sulla scansione dei dati al fine di identificare gruppi di item che appaiono insieme nello stesso basket con una frequenza superiore a una certa soglia (chiamata _supporto_ minimo).

Gli itemset frequenti vengono spesso presentati come regole if-then che prendono il nome di __Association Rules__. La forma di una regola di associazione è del tipo $I \rightarrow J$, dove $I$ è un itemset e $J$ è un item. L'implicazione indica che se un basket contiene tutti gli item di $I$, allora è _verosimile_ che questo basket contenga anche $J$. Il concetto di _verosimiglianza_ viene formalizzato definendo la _confidence_ della regola (rapporto tra il supporto di $I \cup \{J\}$ e il supporto di $I$).

## FP-Growth
La scoperta degli itemset frequenti può essere effettuata attraverso algoritmi come __FP-Growth__. Questo algoritmo codifica il dataset usando una struttura dati compatta che prende il nome di __FP-Tree__ ed estrae gli itemset frequenti direttamente da questa struttura. Un FP-Tree è una rappresentazione compatta dei basket del dataset. Si costruisce leggendo un basket alla volta e mappando ciascuno di questi in un percorso dell'FP_Tree. Diversi basket possono avere item in comune, quindi i loro percorsi possono sovrapporsi. Maggiore è il numero di sovrapposizioni, maggiore è la compressione ottenuta mediante la rappresentazione con l'FP-Tree.

In [40]:
# Carichiamo il dataset
baskets = spark.read.csv("../data/file7.csv", inferSchema=True, header=True)

# Stampiamo qualche elemento
baskets.show(3, truncate=False)

+-------------+----------+---------------+
|Member_number|Date      |itemDescription|
+-------------+----------+---------------+
|1808         |21-07-2015|tropical fruit |
|2552         |05-01-2015|whole milk     |
|2300         |19-09-2015|pip fruit      |
+-------------+----------+---------------+
only showing top 3 rows



Dobbiamo ri-organizzare il nostro dataset in modo da avere dei basket di prodotti. Per fare ciò, usiamo la funzione `groupBy` e `agg` di Spark.

In [41]:
# Raggruppiamo in delle liste tutti gli item acquistati dalla stessa persona nello stesso giorno
baskets = baskets.groupBy("Member_number", "Date").agg(collect_list("itemDescription").alias("items"))

# Rimuoviamo i duplicati da ciascuna lista
baskets = baskets.withColumn("items", array_distinct("items"))

# Stampiamo qualche elemento
baskets.show(5, truncate=False)

+-------------+----------+--------------------------------------------------+
|Member_number|Date      |items                                             |
+-------------+----------+--------------------------------------------------+
|1000         |15-03-2015|[sausage, whole milk, semi-finished bread, yogurt]|
|1000         |24-06-2014|[whole milk, pastry, salty snack]                 |
|1000         |24-07-2015|[canned beer, misc. beverages]                    |
|1000         |25-11-2015|[sausage, hygiene articles]                       |
|1000         |27-05-2015|[soda, pickled vegetables]                        |
+-------------+----------+--------------------------------------------------+
only showing top 5 rows



In [42]:
# Creiamo e addestiamo il modello
fp_growth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.01)
model = fp_growth.fit(baskets)

# Visualizziamo alcuni tra gli itemset frequenti
model.freqItemsets.show(3)

# Visualizziamo alcune regole di associazione
model.associationRules.show(3)

+------------+----+
|       items|freq|
+------------+----+
|      [beef]| 508|
|[white wine]| 175|
| [chocolate]| 353|
+------------+----+
only showing top 3 rows

+------------------+------------+-------------------+------------------+--------------------+
|        antecedent|  consequent|         confidence|              lift|             support|
+------------------+------------+-------------------+------------------+--------------------+
|[other vegetables]|[rolls/buns]|0.08648056923918993|0.7861535586427697|0.010559379803515338|
|[other vegetables]|[whole milk]|0.12151067323481117|0.7694304712706219|0.014836596939116486|
|          [yogurt]|[whole milk]|0.12996108949416343|0.8229402378760761|0.011160863463209249|
+------------------+------------+-------------------+------------------+--------------------+
only showing top 3 rows

